In [ ]:
!pip install pmdarima
from pandas import read_csv
from pandas import datetime
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from pandas.plotting import autocorrelation_plot
from pmdarima.arima import auto_arima
from pmdarima.arima import ADFTest
# pd.show_versions()

In [ ]:
## read input
data = read_csv('/content/input')

## pre-processing
data.set_index('dateRep',inplace=True)
data = data.drop(['day','month','year','deaths','geoId','countryterritoryCode','popData2019','continentExp','Cumulative_number_for_14_days_of_COVID-19_cases_per_100000'], axis=1)
clean_data = data[::-1]
clean_data.to_csv('/content/clean_data' , index=True )

for i in range(0,47480):
  if clean_data.iloc[i,0]<0:
    clean_data.iloc[i,0] = 0

d = data.groupby(['countriesAndTerritories'])
countries_size = d.size().to_frame()
countries_size.to_csv('/content/countries_size' , index=True )

In [ ]:
## model

countries_size = read_csv('/content/countries_size')

# initial ans dataframe
name_list = []
for i in range(0,210):
  ## 159 Russia, 78 Greece, 92 India, 201 USA, 193 Turkey
  if (i==159 or i==78 or i==92 or i==201 or i==193):
    name = countries_size.iloc[i,0]
    name_list.append(name)  
ans = pd.DataFrame(columns=name_list)

# model selection
for i in range(0,210):

  ## 159 Russia, 78 Greece, 92 India, 201 USA, 193 Turkey
  if (i==159 or i==78 or i==92 or i==201 or i==193):
    name = countries_size.iloc[i,0]
    size = int(countries_size.iloc[i,1])

    one_country_data = clean_data[clean_data['countriesAndTerritories'] == name]
    one_country_data = one_country_data.drop(['countriesAndTerritories'], axis=1)

    # adf_test = ADFTest(alpha = 0.05)
    # check = adf_test.should_diff(one_country_data)

    # train = one_country_data[:size-7]
    # valid = one_country_data[-7:]

    train = one_country_data[0:size]
    valid = one_country_data[-7:]

    if (i==0 or i==10 or i==12 or i==16 or i==19 or i==25 or i==45 or i==47 or i==48 or i==59 or i==60 or i==65 or i==67 or i==84 or i==86 or i==87 or i==89 or i==90 or i==96 or i==98 or i==100 or i==103 or i==115 or i==117 or i==118 or i==125 or i==127 or i==133 or i==147 or i==150 or i==152 or i==155 or i==156 or i==167 or i==175 or i==176 or i==182 or i==187 or i==188 or i==193 or i==195 or i==198 or i==204):
      model = auto_arima(train,trace=True,start_p=0,start_q=0,start_P=0,start_Q=0,max_d=10,max_P=10,max_Q =10,information_criterion='bic' )
    elif(i==92 or i==94 or i==99 or i==137 or i==145 or i==166 or i==177 or i==181 or i==184):
      model = auto_arima(train,trace=True)
    else:
      model = auto_arima(train,trace=True,start_p=0,start_q=0,start_P=0,start_Q=0,max_d=10,max_P=10,max_Q =10)

    print(model.summary())

    prediction = pd.DataFrame(model.predict(n_periods=7),index=['0','1','2','3','4','5','6'])

    prediction.columns = ['predicted_cases']
    prediction['predicted_cases'] = prediction['predicted_cases'].round(0).astype('int')
    df = prediction['predicted_cases']

    case_list = []
   
    ## plot
    # print("COVID-19 Forecast")
    # print(name)
    # plt.plot(train, label='Train')
    # plt.plot(valid, label='Valid')
    # plt.plot(prediction, label='Predict')
    # plt.legend(loc='Left corner')
    # plt.show()

    for x in range(0,7):
      case = prediction.iloc[x,0]
      case = int(case)
      if case<0:
        case = 0
      
      ## evaluate function
      # if valid.iloc[x,0]!=0:
      #   error = error + (abs(valid.iloc[x,0]-case)/valid.iloc[x,0])
      
      case_list.append(case) 
    ans[name] = case_list

ans = ans.rename(index={0:'10/9',1:'10/10',2:'10/11',3:'10/12',4:'10/13',5:'10/14',6:'10/15'})

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=2888.611, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=2856.176, Time=0.05 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=2815.723, Time=0.10 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=2886.948, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=2809.221, Time=0.18 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=2809.941, Time=0.24 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=2810.629, Time=0.26 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=2808.635, Time=0.22 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=2810.619, Time=0.34 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=2812.601, Time=0.35 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=2811.224, Time=0.07 sec

Best model:  ARIMA(0,1,2)(0,0,0)[0] intercept
Total fit time: 1.847 seconds
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  

/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


 ARIMA(2,2,2)(0,0,0)[0]             : AIC=5223.022, Time=0.62 sec
 ARIMA(0,2,0)(0,0,0)[0]             : AIC=5482.543, Time=0.01 sec
 ARIMA(1,2,0)(0,0,0)[0]             : AIC=5448.592, Time=0.02 sec
 ARIMA(0,2,1)(0,0,0)[0]             : AIC=inf, Time=0.10 sec
 ARIMA(1,2,2)(0,0,0)[0]             : AIC=inf, Time=0.25 sec
 ARIMA(2,2,1)(0,0,0)[0]             : AIC=inf, Time=0.30 sec


/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/sarimax.py:963: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


 ARIMA(3,2,2)(0,0,0)[0]             : AIC=inf, Time=0.62 sec
 ARIMA(2,2,3)(0,0,0)[0]             : AIC=inf, Time=0.80 sec
 ARIMA(1,2,1)(0,0,0)[0]             : AIC=inf, Time=0.21 sec


...

/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


 ARIMA(5,1,5)(0,0,0)[0] intercept   : AIC=5552.239, Time=1.78 sec


/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tsa/statespace/sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


 ARIMA(4,1,5)(0,0,0)[0] intercept   : AIC=5588.806, Time=1.66 sec
 ARIMA(5,1,5)(0,0,0)[0]             : AIC=5554.273, Time=1.56 sec

Best model:  ARIMA(5,1,5)(0,0,0)[0] intercept
Total fit time: 13.553 seconds
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  283
Model:               SARIMAX(5, 1, 5)   Log Likelihood               -2764.120
Date:                Mon, 12 Oct 2020   AIC                           5552.239
Time:                        10:33:30   BIC                           5595.942
Sample:                             0   HQIC                          5569.765
                                - 283                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------


/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [ ]:
## output

print("Model Prediction:")
print(ans)

ans.to_csv('/content/output' , index=True )

Model Prediction:
       Greece  India  Russia  Turkey  United_States_of_America
10/9      366  75759   11576    1581                     48507
10/10     359  70819   11716    1581                     55395
10/11     361  68283   11943    1581                     49237
10/12     362  68014   12128    1581                     42093
10/13     363  68110   12313    1581                     44803
10/14     364  67518   12486    1581                     47542
10/15     366  66381   12650    1581                     45989
